In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import cv2
import matplotlib.pyplot as plt
# example of loading an image with the Keras API
from keras.preprocessing.image import load_img
from PIL import Image,ImageFile
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf

In [ ]:
import PIL

# loading clean data

In [ ]:
descriptions = None
with open("file.txt",'r') as f:
    descriptions= f.read()
    
json_acceptable_string = descriptions.replace("'","\"")
descriptions = json.loads(json_acceptable_string)

In [ ]:
print(type(descriptions))

In [ ]:
print(descriptions["667626_18933d713e"])

In [ ]:
vocab = set()
for key in descriptions.keys():
    for caption in descriptions[key]:
        vocab.update(caption.split())

In [ ]:
print(len(vocab))

In [ ]:
total_words=[]
for key in descriptions.keys():
    for caption in descriptions[key]:
        caption=caption.split()
        for i in caption:
            total_words.append(i)

In [ ]:
print(len(total_words))

In [ ]:
import collections

counter=collections.Counter(total_words)
freq_cnt = dict(counter)

In [ ]:
for x,y in freq_cnt.items():
    print(x,y)
    

In [ ]:
sorted_freq= sorted(freq_cnt.items(),reverse=True,key=lambda x:int(x[1]))


sorted_freq=[x for x in sorted_freq if x[1]>10]
total_words = [x[0] for x in sorted_freq]

In [ ]:
print(len(total_words))
    

# Image feature extraction

In [ ]:

path="./Images/"
def preprocess_img(img):
    img=cv2.imread(img)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    x = np.asarray(img, dtype=float)
    x=x/224
    x= np.expand_dims(x,axis=0)
    return x


In [ ]:
img=preprocess_img(path+"667626_18933d713e.jpg")
print(img.shape)

In [ ]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))


# model.summary()

In [ ]:

model_new = Model(model.input,model.layers[-2].output)

In [ ]:
img=preprocess_img(path+"667626_18933d713e.jpg")
img_vector=model_new.predict(img)
print(img_vector.shape)

In [ ]:
img_vector.reshape((-1,))
print(img_vector.shape)


In [ ]:
def image_feature_extraction(img):
    img=preprocess_img(img)
    img_vector=model_new.predict(img)
    img_vector=img_vector.reshape((-1,))
    return img_vector
    
    

# train test split

In [ ]:
key=[]
for i in descriptions:
    key.append(i)

In [ ]:
print(len(key))

In [ ]:
train,test=train_test_split(key,test_size=0.2)

In [ ]:
print(len(train))

# image CNN

In [ ]:
image_feature={}
for i in train:
       image_feature[i]=image_feature_extraction(path+i+".jpg")
    
    

In [ ]:

with open("img_features.pkl","wb") as f:
     pickle.dump(image_feature,f)

In [ ]:
#with open('img_features.pkl', 'rb') as f:
  #  data = pickle.load(f)
   # image_feature=data

In [ ]:
image_feature_test={}
for i in test:
    image_feature_test[i]=image_feature_extraction(path+i+".jpg")

In [ ]:
with open("img_test_features.pkl","wb") as f:
     pickle.dump(image_feature_test,f)

In [ ]:
numtoword={}
wordtonum={}

for j,i in enumerate(total_words):
    numtoword[j+1]=i
    wordtonum[i]=j+1
    

In [ ]:
print(len(wordtonum))
print(len(numtoword))
total_len=len(wordtonum)+3

In [ ]:
wordtonum['startseq']=1846
numtoword[1846]='startseq'

In [ ]:
wordtonum['endseq']=1847
numtoword[1847]='endseq'


In [ ]:
maxlen=0
for key in descriptions.keys():
    for i in descriptions[key]:
        maxlen=max(maxlen,len(i.split()))
        

In [ ]:
maxlen+=2


# data input

In [ ]:
def inputdata(image_feature,descriptions,wordtonum,maxlen,batch_size):
    X1,X2, y = [],[],[]
    
    n =0
    while True:
        for key in train.keys():
            n += 1
            
            photo = image_feature[key]
            for desc in descriptions[key] :
                seq=[]
                seq.append(wordtonum["startseq"])
                for word in desc.split():
                      if(word in wordtonum):
                        seq.append(wordtonum[word])
                seq.append(wordtonum["endseq"])
                for i in range(0,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]
                    
                    #0 denote padding word
                    xi = pad_sequences([xi],maxlen=maxlen,value=0,padding='post')[0]
                    yi = to_categorical([yi],num_classes=total_len)[0]
                    
                    X1.append(photo)
                    X2.append(xi)
                    y.append(yi)
                    
                if n==batch_size:
                    yield [[np.array(X1),np.array(X2)],np.array(y)]
                    X1,X2,y = [],[],[]
                    n = 0
         

# embedding 

In [ ]:
f =open("glove.6B.50d.txt",encoding='utf8')

In [ ]:

embedding_index = {}

for line in f:
    values = line.split()
    
    word = values[0]
    word_embedding = np.array(values[1:],dtype='float')
    embedding_index[word] = word_embedding

In [ ]:
f.close()

In [ ]:
embedding_index['and']

In [ ]:
def get_embedding_matrix():
    emb_dim = 50
    matrix = np.zeros((total_len,emb_dim))
    for word,idx in wordtonum.items():
        if embedding_index.get(word) is not None:
            embedding_vector = embedding_index.get(word)
            matrix[idx] = embedding_vector
            
    return matrix

In [ ]:
embedding_matrix = get_embedding_matrix()
embedding_matrix.shape

In [ ]:
embedding_matrix[7]

In [ ]:
np.savetxt('embedding.txt', embedding_matrix, fmt='%f')

# image captioning model

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(maxlen,))
se1 = Embedding(total_len, 50, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(total_len, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
final_model=Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [ ]:
final_model.summary()

In [ ]:
final_model.layers[2].set_weights([embedding_matrix])
final_model.layers[2].trainable = False

In [ ]:
final_model.compile(loss='categorical_crossentropy',optimizer="adam")

# training

In [ ]:
epochs = 20
batch_size = 4
steps = len(train)//6
print(steps)

In [ ]:
for i in range(epochs):
        generator = inputdata(train,image_feature,image_feature_test,descriptions,wordtonum,maxlen,batch_size)
        final_model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
        final_model.save('model_'+str(i)+'.h5')

# prediction

In [ ]:
model = load_model('3_model_9.h5')

In [ ]:

def predict_caption(photo):
    
    in_text = "startseq"
    for i in range(35):
        sequence = [wordtonum[w] for w in in_text.split() if w in wordtonum]
        sequence = pad_sequences([sequence],maxlen=35,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax() #WOrd with max prob always - Greedy Sampling
        word = numtoword[ypred]
        in_text += (' ' + word)
        
        if word == "endseq":
            break
    
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [ ]:
img=preprocess_img(path+"755326139_ee344ece7b.jpg")

In [ ]:
img_vector=model_new.predict(img)
print(img_vector.shape)

In [ ]:
predict_caption(img_vector)